In [1]:
import pygame
import heapq
from collections import deque

# oyun penceresi ayarları
width, height = 280, 280
cellSize = 30 
WIN = pygame.display.set_mode((width, height))

pygame 2.5.2 (SDL 2.28.3, Python 3.11.5)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
# oyun haritası
labirent_map = [
    ['#','#','#','#','#','#','#','#','#'],
    ['#','S','.','.','.','.','.','.','#'],
    ['#','.','#','#','#','.','#','#','#'],
    ['#','.','.','.','#','.','.','#','#'],
    ['#','#','#','.','#','.','.','.','#'],
    ['#','.','.','.','.','.','#','.','#'],
    ['#','E','#','#','#','#','#','.','#'],
    ['#','.','.','.','.','.','.','.','#'],
    ['#','#','#','#','#','#','#','#','#']
]

In [3]:
# başlangıç ve bitişin konumu
start, end = None, None
for y, row in enumerate(labirent_map):
    for x, cell in enumerate(row):
        if cell == 'S':
            start = (x, y)
        elif cell == 'E':
            end = (x, y)

if start is None or end is None:
    raise ValueError("Başlangıç veya bitiş noktaları bulunamadı.")

In [4]:
# A* algoritması
def heuristic_calculate(a, b):
    return abs(b[0] - a[0]) + abs(b[1] - a[1])

def a_star_shortest(labirent_map, start, end):
    queue = []
    heapq.heappush(queue, (0, start))
    paths = {start: None}
    g_score = {start: 0}
    f_score = {start: heuristic_calculate(start, end)}

    while queue:
        current = heapq.heappop(queue)[1]

        if current == end:
            path = []
            while current is not None:
                path.append(current)
                current = paths[current]
            path.reverse()
            return path

        for zx, zy in ((-1, 0), (1, 0), (0, -1), (0, 1)):
            x, y = current[0] + zx, current[1] + zy
            if (0 <= x < len(labirent_map[0])) and (0 <= y < len(labirent_map)) and (labirent_map[y][x] != '#'):
                new_g_score = g_score[current] + 1
                if (x, y) not in g_score or new_g_score < g_score[(x, y)]:
                    g_score[(x, y)] = new_g_score
                    f_score[(x, y)] = new_g_score + heuristic_calculate((x, y), end)
                    paths[(x, y)] = current
                    heapq.heappush(queue, (f_score[(x, y)], (x, y)))
                    
path = a_star_shortest(labirent_map, start, end)

In [5]:
# oyun döngüsü
done = False
draw_path = False 
while not done:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            done = True
        elif event.type == pygame.KEYDOWN:  
            if event.key == pygame.K_SPACE: 
                draw_path = True

    # oyun penceresini güncelleme
    WIN.fill((255, 255, 255))  
    for y in range(len(labirent_map)):
        for x in range(len(labirent_map[y])):
            if labirent_map[y][x] == '#':
                pygame.draw.rect(WIN, (0, 0, 0), pygame.Rect(x * cellSize, y * cellSize, cellSize, cellSize))  
            elif labirent_map[y][x] == 'S':
                pygame.draw.rect(WIN, (255, 0, 0), pygame.Rect(x * cellSize, y * cellSize, cellSize, cellSize)) 
            elif labirent_map[y][x] == 'E':
                pygame.draw.rect(WIN, (255, 255, 0), pygame.Rect(x * cellSize, y * cellSize, cellSize, cellSize))  
            elif draw_path and (x, y) in path:
                pygame.draw.rect(WIN, (0, 255, 0), pygame.Rect(x * cellSize, y * cellSize, cellSize, cellSize))  
    pygame.display.update()

pygame.quit()